<a href="https://colab.research.google.com/github/ShubhranshuPattnaik/summarization-t5-small/blob/main/summarization_t5_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install datasets evaluate transformers rouge-score nltk

In [18]:
from huggingface_hub import notebook_login
notebook_login()

In [19]:
!apt install git-lfs

In [20]:
import transformers

print(transformers.__version__)
model_checkpoint = "t5-small"
from datasets import load_dataset
from evaluate import load

raw_datasets = load_dataset("xsum")
metric = load("rouge")

In [22]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [23]:
raw_datasets["train"] = raw_datasets["train"].select(range(500))


show_random_elements(raw_datasets["train"])

metric
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

,document,summary,id
0,"Economic nationalists like senior adviser Steve Bannon view withdrawal as visible way for the US to demonstrate that it's putting its own economic interests ahead of the concerns of the ""international community"".\nMovement conservatives, including Environmental Protection Agency head Scott Pruitt, are keen to strike a blow to the environmentalist coalition, which they view as creeping socialism cloaked in an earth-friendly guise.\nMeanwhile Mr Trump's working-class supporters - particularly those in the economically distressed coal-producing regions of West Virginia, Ohio and Pennsylvania that delivered an Electoral College victory to the president - are more concerned about jobs and their way of life, rather than the distant, amorphous threat of rising sea levels or shifting climate patterns.\nIvanka Trump, along with her husband (and senior White House adviser) Jared Kushner, Secretary of State Rex Tillerson and Secretary of Defence James Mattis, were reportedly strong advocates for remaining in the agreement - for a range of environmental, diplomatic and national security reasons.\nThe president's daughter even arranged for her father to meet politician-turned-activist Al Gore to discuss the issue during the presidential transition.\nThe efforts always figured to be an uphill battle, however, given the issues and interests that formed the heart of Mr Trump's presidential campaign.\nMr Trump once notably called global warming a ""hoax"" perpetrated by the Chinese, but over the course of last year's campaign he was largely silent on environmental issues.\nHe spoke early and often about jobs, the economy and government overregulation, however, and will likely frame any move to abandon the Paris agreement as evidence that his presidency is taking action to put more money in American pockets.\nIf such a move angers the international elite, US liberals and media talking heads, so much the better.",While a presidential decision to pull out of the Paris Agreement would roil the international community and frustrate White House advocates of addressing climate change - including Donald Trump's own daughter - there are formidable forces within the administration in favour of such a move.,40054265
1,"Prosecutors accuse Fifa president Sepp Blatter of making a ""disloyal payment"" of $2m (Â£1.6m) to Mr Platini.\nMr Platini has provided information to the investigation but said he did so as a witness.\nBoth men deny any wrongdoing.\nMr Platini has said he is still determined to run for Fifa president once Mr Blatter steps down.\nAsked whether criminal proceedings had been opened against him, Mr Platini said: ""Absolutely not. I was heard last week by the Swiss Authorities only as a person providing information and I cooperated fully.""\nBut this was contradicted by the Swiss attorney general, Michael Lauber, who said: ""We didn't interview Mr Platini as a witness, that's not true. We investigated against him in between as a witness and an accused person.""\nMr Lauber also said he was prepared to search the Uefa headquarters as part of the investigation.\nThe allegations centre on a payment made to Mr Platini in 2011 at Fifa's expense for work he completed almost a decade earlier.\nIt came just two months before Uefa gave its backing to Blatter's 2011 presidential campaign, leading some to question the timing of it.\nMr Platini said he was told when starting the work that Fifa would not be able to pay him all the money because of its ""financial situation"".\nBut Fifa accounts show a revenue surplus of around $83m (Â£55m) from 1999-2002, when Mr Platini undertook the work as a technical adviser.\nFifa was thrown into turmoil this year when the US and Swiss authorities launched separate investigations into corruption at the body.\nMr Blatter is also suspected of signing a contract that was ""unfavourable to Fifa"". He is due to stand down next year, but says he will carry on in the role despite the criminal proceed

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

In [24]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer("Hello, this one sentence!")
tokenizer(["Hello, this one sentence!", "This is another sentence."])
print(tokenizer(text_target=["Hello, this one sentence!", "This is another sentence."]))

if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""
max_input_length = 1024
max_target_length = 128

In [25]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

preprocess_function(raw_datasets['train'][:2])
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True, batch_size=16)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/11332 [00:00<?, ? examples/s]

Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [26]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


batch_size = 8
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=200,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    result = {key: value * 100 for key, value in result.items()}


    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [28]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [31]:
trainer.train()


Step,Training Loss,Validation Loss


TrainOutput(global_step=31, training_loss=3.5053208874117945, metrics={'train_runtime': 28.7815, 'train_samples_per_second': 17.372, 'train_steps_per_second': 1.077, 'total_flos': 117960953561088.0, 'train_loss': 3.5053208874117945, 'epoch': 0.9841269841269841})

In [37]:
import nltk
nltk.download('punkt')

predictions = trainer.predict(tokenized_datasets["validation"])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/Shubhranshu21/t5-small-finetuned-xsum/commit/b641bbb5391831425d6d0d6f87a9678dedf81ee9', commit_message='End of training', commit_description='', oid='b641bbb5391831425d6d0d6f87a9678dedf81ee9', pr_url=None, pr_revision=None, pr_num=None)

In [41]:
!pip install gradio


In [42]:
import gradio as gr

def summarize_text(input_text):
    summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
    summary = summarizer(input_text, max_length=128, min_length=30, do_sample=False)
    return summary[0]['summary_text']

interface = gr.Interface(
    fn=summarize_text,
    inputs="text",
    outputs="text",
    title="Text Summarization with Fine-Tuned Model",
    description="Enter text to get a summary using the fine-tuned T5 model."
)


interface.launch()